Under the condition where epoch = 100, compare three kinds of convnets using different training ways:

1. Firstly train digits 0~4, then 5~9
2. Firstly train digits 0~4, then 0~9
3. Firstly train digits 0~9

Compare way:

1. curves' shape
2. accuracy on test sets
3. **[Note]** Conclusions must be based on means of results from multiple experients, 5 at least.

You should commit:

1. Source code
2. A PDF report with details of experiments, including details of methods, results in graphs and conclusion.

 --------------------------------------------------------------

# 1. ConvNet_0 [0~4 -> 5~9]

## 1.1 train and test

In [ ]:
# define class

In [ ]:
# train using 0~4

In [ ]:
# train using 5~9

In [ ]:
# test

## 1.2 details

blabla...

# 2. ConvNet_1 [0~4 -> 0~9]

## 2.1 train and test

In [ ]:
# define class

In [ ]:
# train using 0~4

In [ ]:
# train using 0~9

In [ ]:
# test

## 2.2 details

blabla...

# 3. ConvNet_2 [0~9]

## 3.1 train and test

In [ ]:
# define class

In [ ]:
# train using 0~9

In [ ]:
# test

## 3.2 details

blabla...